In [1]:
import pandas as pd 
jeopardy=pd.read_csv("data/jeopardy.csv")
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [2]:
jeopardy.columns=jeopardy.columns.str.strip()
jeopardy.columns



Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')

In [3]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Show Number  19999 non-null  int64 
 1   Air Date     19999 non-null  object
 2   Round        19999 non-null  object
 3   Category     19999 non-null  object
 4   Value        19999 non-null  object
 5   Question     19999 non-null  object
 6   Answer       19999 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [4]:
import re
def normalize_text(input):
    input=input.lower()
    input=re.sub('\W',' ',input)
    return input

In [5]:
jeopardy['Value'].unique()

array(['$200', '$400', '$600', '$800', '$2,000', '$1000', '$1200',
       '$1600', '$2000', '$3,200', 'None', '$5,000', '$100', '$300',
       '$500', '$1,000', '$1,500', '$1,200', '$4,800', '$1,800', '$1,100',
       '$2,200', '$3,400', '$3,000', '$4,000', '$1,600', '$6,800',
       '$1,900', '$3,100', '$700', '$1,400', '$2,800', '$8,000', '$6,000',
       '$2,400', '$12,000', '$3,800', '$2,500', '$6,200', '$10,000',
       '$7,000', '$1,492', '$7,400', '$1,300', '$7,200', '$2,600',
       '$3,300', '$5,400', '$4,500', '$2,100', '$900', '$3,600', '$2,127',
       '$367', '$4,400', '$3,500', '$2,900', '$3,900', '$4,100', '$4,600',
       '$10,800', '$2,300', '$5,600', '$1,111', '$8,200', '$5,800',
       '$750', '$7,500', '$1,700', '$9,000', '$6,100', '$1,020', '$4,700',
       '$2,021', '$5,200', '$3,389'], dtype=object)

In [51]:
def normalize_digit(input):
    input=input.lower()
    try:
        input=int(re.sub('\W','',input))
    except Exception:
        input=0
    return input

In [45]:
jeopardy['clean_question']=jeopardy['Question'].apply(normalize_text)

In [46]:
jeopardy['clean_question']

19325    adventurous 26th president  he was 1st to ride...
19307         why april 28th was a bad day for capt  bligh
19312    since  27  stars have made good impressions at...
19299    sensitive mart crowley treatment of gays march...
19274                                     formerly formosa
                               ...                        
1916     he started a book business from his home in 18...
1917     peter  peter was an eater of this  he kept his...
1960                          fred armisen as barack obama
1918     this largest alaskan city lies at the head of ...
1922     in 1997 tyco international moved to this u k  ...
Name: clean_question, Length: 19999, dtype: object

In [47]:
jeopardy['clean_answer']=jeopardy['Answer'].apply(normalize_text)

In [48]:
jeopardy['clean_answer']

19325                     theodore roosevelt
19307    the day of the mutiny on the bounty
19312                 mann s chinese theatre
19299                   the boys in the band
19274                                 taiwan
                        ...                 
1916                      charles m   barnes
1917                                 pumpkin
1960                     saturday night live
1918                               anchorage
1922                                 bermuda
Name: clean_answer, Length: 19999, dtype: object

In [52]:
jeopardy['value_clean']=jeopardy['Value'].apply(normalize_digit)

In [53]:
jeopardy['value_clean']

19325       0
19307     400
19312     600
19299     500
19274     100
         ... 
1916      200
1917      200
1960     1200
1918      200
1922      400
Name: value_clean, Length: 19999, dtype: int64

In [13]:
jeopardy['Air Date']=pd.to_datetime(jeopardy['Air Date'])

In [14]:
jeopardy['Air Date']

0       2004-12-31
1       2004-12-31
2       2004-12-31
3       2004-12-31
4       2004-12-31
           ...    
19994   2000-03-14
19995   2000-03-14
19996   2000-03-14
19997   2000-03-14
19998   2000-03-14
Name: Air Date, Length: 19999, dtype: datetime64[ns]

In [15]:
def count_matches(row):
    split_answer=row['clean_answer'].split()
    split_question=row['clean_question'].split()
    match_count=0
    if "the" in split_answer:
        split_answer.remove('the')
    if len(split_answer)==0 :
        return 0
    for v in split_answer:
        if v in split_question:
            match_count+=1
    return match_count/len(split_answer)
        

In [16]:
jeopardy["answer_in_question"] = jeopardy.apply(count_matches,axis=1)

In [26]:
mean_answer=jeopardy["answer_in_question"].mean()*100

### Recycled questions

In [27]:
print('''On average, the answer only makes up for about {:.2f}% of the question. This isn't a huge number, and means that we probably can't just hope that hearing a question will enable us to figure out the answer. We'll probably have to study.'''.format(mean_answer))

On average, the answer only makes up for about 6.29% of the question. This isn't a huge number, and means that we probably can't just hope that hearing a question will enable us to figure out the answer. We'll probably have to study.


In [57]:
question_overlap=[]
terms_used=set()
jeopardy.sort_values('Air Date',inplace=True)
for i,row in jeopardy.iterrows():
    split_question=row['clean_question'].split(" ")
    split_question=[q for q in split_question if len(q)>5]
    match_count=0
    for word in split_question:
        if word in terms_used :
            match_count+=1
        terms_used.add(word)
    if len(split_question)>0:
        match_count/=len(split_question)
    question_overlap.append(match_count)
jeopardy['question_overlap']=question_overlap
mean_question=jeopardy['question_overlap'].mean()*100
    

    

### Low value vs high value questions

In [32]:
print('''There is about {:.2f}% overlap between terms in new questions and terms in old questions. This only looks at a small set of questions, and it doesn't look at phrases, it looks at single terms. This makes it relatively insignificant, but it does mean that it's worth looking more into the recycling of questions.'''.format(mean_question))

There is about 72.17% overlap between terms in new questions and terms in old questions. This only looks at a small set of questions, and it doesn't look at phrases, it looks at single terms. This makes it relatively insignificant, but it does mean that it's worth looking more into the recycling of questions.


In [35]:
def determine_value(row):
    if row['value_clean']>800:
        value=1
    else: 
        value=0
    return value

jeopardy['high_value']=jeopardy.apply(determine_value,axis=1)

In [36]:
jeopardy['high_value'].value_counts(dropna=False)

0    14265
1     5734
Name: high_value, dtype: int64

In [37]:
def count_usage(term):
    low_count=0
    high_count=0
    for i,row in jeopardy.iterrows():
        if term in row['clean_question'].split(" "):
            if row['high_value']==1:
                high_count+=1
            else:
                low_count+=1
    return high_count,low_count


In [38]:
from random import choice

terms_used_list = list(terms_used)
comparison_terms = [choice(terms_used_list) for _ in range(10)]

In [40]:
observed_expected=[]
for term in comparison_terms:
    observed_expected.append(count_usage(term))

observed_expected

[(2, 1),
 (0, 1),
 (1, 1),
 (8, 16),
 (1, 0),
 (1, 2),
 (2, 1),
 (2, 3),
 (0, 1),
 (0, 1)]

In [59]:
from scipy.stats import chisquare
high_value_count = jeopardy[jeopardy["high_value"] == 1].shape[0]
low_value_count = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []
for obs in observed_expected:
    total = sum(obs)
    total_prop = total / jeopardy.shape[0]
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = [obs[0], obs[1]]
    expected = [high_value_exp, low_value_exp]
    chi_squared.append(chisquare(observed, expected))
    
chi_squared

[Power_divergenceResult(statistic=2.1177104383031944, pvalue=0.14560406868263753),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.4448774816612795, pvalue=0.5047776487545996),
 Power_divergenceResult(statistic=0.255049337875229, pvalue=0.6135419116911882),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.03188116723440362, pvalue=0.8582887163235293),
 Power_divergenceResult(statistic=2.1177104383031944, pvalue=0.14560406868263753),
 Power_divergenceResult(statistic=0.3137668167849311, pvalue=0.5753778622944691),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469)]